## Part 1: Seq2Seq without Attention

1. Importing Necessary Libraries

In [1]:
import tensorflow as tf
tf.config.optimizer.set_jit(True)

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [2]:
import os
import random
import json
import pickle
import numpy as np
from tqdm import tqdm
import nltk
nltk.download('punkt')

import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense, TimeDistributed, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2. Configuration

In [3]:
with open('config.json', 'r') as f:
    config = json.load(f)

eng_vocab_size = config['eng_vocab_size']
fra_vocab_size = config['fra_vocab_size']

EMBEDDING_DIM = 100
HIDDEN_UNITS = 256
STACKED_LAYERS = 2
BATCH_SIZE = 256
EPOCHS = 30
LEARNING_RATE = 0.001

3. Load Preprocessed Data & Embeddings

In [4]:
data = np.load('data.npz', allow_pickle=True)
encoder_input_data = np.array(data['encoder_input_data'], dtype=np.int32)
decoder_input_data = np.array(data['decoder_input_data'], dtype=np.int32)
decoder_target_data = np.array(data['decoder_target_data'], dtype=np.int32)

if decoder_target_data.ndim == 3 and decoder_target_data.shape[-1] == 1:
    decoder_target_data = np.squeeze(decoder_target_data, axis=-1)

with open('en_embedding_matrix.pkl', 'rb') as f:
    en_embedding_matrix = pickle.load(f)

val_size = int(0.1 * len(encoder_input_data))

def create_bucketed_dataset(enc, dec, tgt, batch_size):
    def gen():
        for x, y, z in zip(enc, dec, tgt):
            yield {"encoder_input": x, "decoder_input": y, "decoder_target": z}

    output_signature = {
        "encoder_input": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        "decoder_input": tf.TensorSpec(shape=(None,), dtype=tf.int32),
        "decoder_target": tf.TensorSpec(shape=(None,), dtype=tf.int32),
    }

    dataset = tf.data.Dataset.from_generator(gen, output_signature=output_signature)

    def map_fn(sample):
        return (sample["encoder_input"], sample["decoder_input"]), sample["decoder_target"]

    def bucket_by_seq_len(example):
        enc_len = tf.shape(example["encoder_input"])[0]
        return tf.cast(enc_len // 10, tf.int64)  # bucket keys

    dataset = dataset.apply(
        tf.data.experimental.group_by_window(
            key_func=bucket_by_seq_len,
            reduce_func=lambda key, ds: ds.map(map_fn).padded_batch(
                batch_size,
                padded_shapes=(([None], [None]), [None]),
                padding_values=((0, 0), 0),
                drop_remainder=True
            ),
            window_size=batch_size
        )
    )

    return dataset.prefetch(tf.data.AUTOTUNE)


train_dataset = create_bucketed_dataset(
    encoder_input_data[:-val_size],
    decoder_input_data[:-val_size],
    decoder_target_data[:-val_size],
    batch_size=48
)

val_dataset = create_bucketed_dataset(
    encoder_input_data[-val_size:],
    decoder_input_data[-val_size:],
    decoder_target_data[-val_size:],
    batch_size=48
)


Instructions for updating:
Use `tf.data.Dataset.group_by_window(...)`.


4. Build Seq2Seq Model

In [5]:
enc_in = Input(shape=(None,), name='encoder_inputs')
x = Embedding(eng_vocab_size, EMBEDDING_DIM, weights=[en_embedding_matrix], trainable=True)(enc_in)

for i in range(STACKED_LAYERS - 1):
    x = Bidirectional(LSTM(HIDDEN_UNITS, return_sequences=True))(x)

encoder_output, fh, fc, bh, bc = Bidirectional(
    LSTM(HIDDEN_UNITS, return_state=True)
)(x)

state_h = Concatenate()([fh, bh])
state_c = Concatenate()([fc, bc])

dec_in = Input(shape=(None,), name='decoder_inputs')
y = Embedding(fra_vocab_size, EMBEDDING_DIM, trainable=True)(dec_in)

decoder_lstm = LSTM(HIDDEN_UNITS * 2, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(y, initial_state=[state_h, state_c])
decoder_dense = TimeDistributed(Dense(fra_vocab_size, activation='softmax', dtype='float32'))
final_output = decoder_dense(decoder_output)

model = Model([enc_in, dec_in], final_output)

def masked_sparse_accuracy(y_true, y_pred):
    y_pred_labels = tf.argmax(y_pred, axis=-1, output_type=tf.int32)
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    correct = tf.cast(tf.equal(y_true, y_pred_labels), tf.float32) * mask
    return tf.reduce_sum(correct) / tf.reduce_sum(mask)

model.compile(
    optimizer=Adam(LEARNING_RATE),
    loss='sparse_categorical_crossentropy',
    metrics=[masked_sparse_accuracy]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, None, 100) │  2,914,200 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, None, 512) │    731,136 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ [(None, 512),     │  1,574,912 │ bidirectional[0]… │
│ (Bidirectional)     │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 100) │  5,088,700 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ bidirectional_1[… │
│ (Concatenate)       │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 512)       │          0 │ bidirectional_1[… │
│ (Concatenate)       │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, None,     │  1,255,424 │ embedding_1[0][0… │
│                     │ 512), (None,      │            │ concatenate[0][0… │
│                     │ 512), (None,      │            │ concatenate_1[0]… │
│                     │ 512)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, None,      │ 26,105,031 │ lstm_2[0][0]      │
│ (TimeDistributed)   │ 50887)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 37,669,403 (143.70 MB)

 Trainable params: 37,669,403 (143.70 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
for (inp, out) in train_dataset.take(1):
    print("Encoder Input:", inp[0].shape)
    print("Decoder Input:", inp[1].shape)
    print("Target:", out.shape)


Encoder Input: (48, 55)
Decoder Input: (48, 57)
Target: (48, 57)


4. Training with Early Stopping

In [7]:
checkpoint = ModelCheckpoint(
    filepath='lstm_no_attention.h5',
    save_best_only=True,
    monitor='val_loss',
)

earlystop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    min_delta=0.001,
    restore_best_weights=True
)

steps_per_epoch = len(encoder_input_data[:-val_size]) // BATCH_SIZE
validation_steps = len(encoder_input_data[-val_size:]) // BATCH_SIZE

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint, earlystop],
    verbose=1,
)

Epoch 1/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 1.8920 - masked_sparse_accuracy: 0.0914

703/703 ━━━━━━━━━━━━━━━━━━━━ 131s 164ms/step - loss: 1.8909 - masked_sparse_accuracy: 0.0915 - val_loss: 0.9090 - val_masked_sparse_accuracy: 0.1985
Epoch 2/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.8693 - masked_sparse_accuracy: 0.2149

703/703 ━━━━━━━━━━━━━━━━━━━━ 112s 159ms/step - loss: 0.8692 - masked_sparse_accuracy: 0.2149 - val_loss: 0.7010 - val_masked_sparse_accuracy: 0.2683
Epoch 3/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.6658 - masked_sparse_accuracy: 0.2888

703/703 ━━━━━━━━━━━━━━━━━━━━ 140s 199ms/step - loss: 0.6658 - masked_sparse_accuracy: 0.2889 - val_loss: 0.5772 - val_masked_sparse_accuracy: 0.3567
Epoch 4/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.5494 - masked_sparse_accuracy: 0.3726

703/703 ━━━━━━━━━━━━━━━━━━━━ 112s 160ms/step - loss: 0.5493 - masked_sparse_accuracy: 0.3726 - val_loss: 0.4997 - val_masked_sparse_accuracy: 0.4139
Epoch 5/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 0.4815 - masked_sparse_accuracy: 0.4256

...

703/703 ━━━━━━━━━━━━━━━━━━━━ 142s 202ms/step - loss: 0.1925 - masked_sparse_accuracy: 0.6767 - val_loss: 0.2805 - val_masked_sparse_accuracy: 0.6144
Epoch 20/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 135s 192ms/step - loss: 0.1827 - masked_sparse_accuracy: 0.6887 - val_loss: 0.2833 - val_masked_sparse_accuracy: 0.6164
Epoch 21/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.1708 - masked_sparse_accuracy: 0.7017

703/703 ━━━━━━━━━━━━━━━━━━━━ 109s 155ms/step - loss: 0.1708 - masked_sparse_accuracy: 0.7017 - val_loss: 0.2794 - val_masked_sparse_accuracy: 0.6202
Epoch 22/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.1610 - masked_sparse_accuracy: 0.7146

703/703 ━━━━━━━━━━━━━━━━━━━━ 144s 205ms/step - loss: 0.1610 - masked_sparse_accuracy: 0.7146 - val_loss: 0.2756 - val_masked_sparse_accuracy: 0.6237
Epoch 23/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.1540 - masked_sparse_accuracy: 0.7266

703/703 ━━━━━━━━━━━━━━━━━━━━ 110s 157ms/step - loss: 0.1540 - masked_sparse_accuracy: 0.7266 - val_loss: 0.2739 - val_masked_sparse_accuracy: 0.6296
Epoch 24/30
235/703 ━━━━━━━━━━━━━━━━━━━━ 1:06 143ms/step - loss: 0.1469 - masked_sparse_accuracy: 0.7342

703/703 ━━━━━━━━━━━━━━━━━━━━ 82s 116ms/step - loss: 0.1463 - masked_sparse_accuracy: 0.7343 - val_loss: 0.2732 - val_masked_sparse_accuracy: 0.6304
Epoch 25/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 104s 148ms/step - loss: 0.1439 - masked_sparse_accuracy: 0.7404 - val_loss: 0.2737 - val_masked_sparse_accuracy: 0.6301
Epoch 26/30
703/703 ━━━━━━━━━━━━━━━━━━━━ 104s 148ms/step - loss: 0.1385 - masked_sparse_accuracy: 0.7504 - val_loss: 0.2786 - val_masked_sparse_accuracy: 0.6312


In [10]:
start_token = '<sos>'
end_token = '<eos>'

with open('fra_tokenizer.pkl', 'rb') as f:
    fra_tokenizer = pickle.load(f)

from nltk.translate.bleu_score import corpus_bleu

def tokens_to_sentence(token_ids, tokenizer, end_token):
    words = []
    for t in token_ids:
        if t == end_token:
            break
        word = tokenizer.index_word.get(t, '')  # or use your vocab mapping
        if word == '':
            continue
        words.append(word)
    return words

import numpy as np
import tensorflow as tf

def beam_search_decode(
    model,
    encoder_input_seq,
    start_token,
    end_token,
    fra_vocab_size,
    beam_width=3,
    max_decoder_seq_length=50,
):
    start_seq = [start_token]
    beam = [(start_seq, 0.0)]
    for _ in range(max_decoder_seq_length):
        all_candidates = []
        for seq, score in beam:
            if seq[-1] == end_token:
                all_candidates.append((seq, score))
                continue
            enc_input = encoder_input_seq
            dec_input = np.array(seq)[np.newaxis, :]
            preds = model.predict([enc_input, dec_input], verbose=0)
            next_token_logits = preds[0, -1, :]
            log_probs = np.log(next_token_logits + 1e-9)
            top_tokens = np.argsort(log_probs)[-beam_width:]
            for t in top_tokens:
                candidate_seq = seq + [int(t)]
                candidate_score = score + log_probs[t]
                all_candidates.append((candidate_seq, candidate_score))
        ordered = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)
        beam = ordered[:beam_width]
        if all(seq[-1] == end_token for seq, _ in beam):
            break
    best_sequence = beam[0][0]
    return best_sequence


def calculate_bleu_score(
    model,
    encoder_input_data,
    decoder_target_data,
    fra_tokenizer,
    start_token_id,
    end_token_id,
    beam_width=3,
    max_decoder_seq_length=50,
    num_samples=100,
):
    references = []
    hypotheses = []

    for i in range(num_samples):
        enc_seq = encoder_input_data[i : i + 1]
        target_seq = decoder_target_data[i]
        pred_seq = beam_search_decode(
            model,
            enc_seq,
            start_token=start_token_id,
            end_token=end_token_id,
            fra_vocab_size=len(fra_tokenizer.word_index) + 1,
            beam_width=beam_width,
            max_decoder_seq_length=max_decoder_seq_length,
        )
        pred_sentence = tokens_to_sentence(pred_seq[1:], fra_tokenizer, end_token_id)
        ref_sentence = tokens_to_sentence(target_seq, fra_tokenizer, end_token_id)
        references.append([ref_sentence])
        hypotheses.append(pred_sentence)

        if i % 10 == 0:
            print(f"Sample {i} | Pred: {' '.join(pred_sentence)} | Ref: {' '.join(ref_sentence)}")
    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score

bleu = calculate_bleu_score(
    model,
    encoder_input_data[-100:],
    decoder_target_data[-100:],
    fra_tokenizer,
    start_token_id=fra_tokenizer.word_index.get(start_token),
    end_token_id=fra_tokenizer.word_index.get(end_token),
    max_decoder_seq_length=50,
    num_samples=100
)

print(f"BLEU score: {bleu:.4f}")


<ipython-input-10-4174476471>:44: RuntimeWarning: divide by zero encountered in log
  log_probs = np.log(next_token_logits + 1e-9)


Sample 0 | Pred: tom est à j'attendrai | Ref: tom est en faveur.
Sample 10 | Pred: il ai a seul. raison de avec peur ? | Ref: il ai a seul. raison de faim. connais fenêtre,
Sample 20 | Pred: je pense que tu cet demain. pensait chose | Ref: je pense que vous faites demain. pensait chose
Sample 30 | Pred: je tenu que vous espion pas plus part | Ref: je tenu que tu grappe. pas plus part
Sample 40 | Pred: les veux-tu fit des gagné | Ref: les veux-tu fit des manger « ça
Sample 50 | Pred: je ne depuis vraiment pas que vous chanter. le projet de sans | Ref: mourir. vraiment que vous l'as ce embrassés
Sample 60 | Pred: type ? c'était tout | Ref: éponge pu tout
Sample 70 | Pred: votre français est cabane | Ref: ton français est cabane
Sample 80 | Pred: boston. petit. | Ref: pins. êtes-vous vous crois
Sample 90 | Pred: il ne n'arrive pas ma l'arbre | Ref: il ne n'arrive pas ma l'arbre
BLEU score: 0.2880
